In [1]:
import pandas as pd
from pathlib import Path
import numpy as np
import os
import matplotlib.pyplot as plt
import scipy.signal as signal
import imageio as io
from tqdm import tqdm
import pickle, re
import cv2
font = cv2.FONT_HERSHEY_SIMPLEX
rgb = (255, 255, 255)

import sys
sys.path.insert(1, '../numphly')
import numphly as nly
import tracker_decoder as tkr_dec
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

import platform
print(platform.python_version())

3.10.14


In [2]:
exp_target = 'R72'
gtype_name = ['R72A10xTRPA1','R72A10xW1118','W1118xTRPA1']
sex_name = ['female','male']
h5_dir = '../../FlyBehaviorSet/DLC_tracking/'+exp_target
output_fileloc = '../../FlyBehaviorSet/'
OG_recording_folder= 'original_recording_for_tracking/'+exp_target
converted_folder = 'Converted/R72_videos/'
cropped_recording_folder= 'cropped_recording_for_tracking'  



dir_MCAM = '../DatastreamLine_MCAM_Tracking/'
scorer_floor  = 'DLC_resnet152_floor_topview_12label_1uniqueJan4shuffle1_100000'
# scorer_wall   = 'DLC_resnet152_wall_sideview_7labelsJan1shuffle1_100000'
scorer_wall   = 'DLC_resnet152_wall_sideview_7labelsJan1shuffle1_90000'
# scorer_cross  = 'DLC_resnet152_wall_cross_section_view_8labelFeb19shuffle1_100000'
scorer_cross  = 'DLC_resnet152_wall_cross_section_view_8labelFeb19shuffle1_30000'

pcutoff = 0
frame_n = 126000
frame_rate = 60

frame_n_e = frame_rate*60*30

color_BP = [(0,0,255),(0,255,0),(255,0,0)]
arena_parameters = {
    'well_radius': 8.4,  # mm
    'food_cup_radius': 3.5,  # mm
}

experiments_index  = np.load('experiments_index.npy')
fileloc_set        = np.load('experiments_directory.npy')
num_file = len(fileloc_set)
videofile_set      = np.load('experiments_well_setup.npy')
num_well = len(videofile_set)
experiments_mscore = np.load('experiments_mscore.npy',allow_pickle='TRUE').item()
crop_h = np.load('crop_h.npy')
print(len(fileloc_set),frame_n_e)

15 108000


In [3]:
fileloc = fileloc_set[0]
experiment_index = experiments_index[0]
well = 'v_'+experiment_index+'_'+videofile_set[0]
numBP_floor = 12
numBP_wall  = 7
numBP_cross = 8
numUniq = 1

hfile_floor = h5_dir+'/'+well+scorer_floor+'_el_filtered.h5'
Dataframe = pd.read_hdf(os.path.join(hfile_floor))
bodyparts = Dataframe.columns.get_level_values(2)[0:3*numBP_floor];
bodyparts_floor = (bodyparts[0::3])

uniqueID = Dataframe.columns.get_level_values(1)[0:3*numBP_floor*3];
uniqueID = (uniqueID[0::3*numBP_floor])[2]
paint = Dataframe.columns.get_level_values(2)[0:3*numBP_floor*3];
paint = paint[3*numBP_floor*2]

hfile_wall = h5_dir+'/'+well+scorer_wall+'_el_filtered.h5'
Dataframe = pd.read_hdf(os.path.join(hfile_wall))
bodyparts = Dataframe.columns.get_level_values(2)[0:3*numBP_wall];
bodyparts_wall = (bodyparts[0::3])

hfile_cross = h5_dir+'/'+well+scorer_cross+'_el_filtered.h5'
Dataframe = pd.read_hdf(os.path.join(hfile_cross))
bodyparts = Dataframe.columns.get_level_values(2)[0:3*numBP_cross];
bodyparts_cross = (bodyparts[0::3])


animalID = Dataframe.columns.get_level_values(1)[0:3*numBP_cross*3];
animalID = (animalID[0::3*numBP_cross])#[0:2]
n_ID = len(animalID)

# numBP_floor = 5
# numBP_wall  = 4
# numBP_cross = 4
# bodyparts_floor = bodyparts_floor[[0,1,2,3,4]]
# bodyparts_wall  = bodyparts_wall[[0,1,2,3]]
# bodyparts_cross = bodyparts_cross[[0,3,4,7]]

numBP_floor = 3
numBP_wall  = 3
numBP_cross = 4
bodyparts_floor = bodyparts_floor[[0,3,4]]
bodyparts_wall  = bodyparts_wall[[1,2,3]]
bodyparts_cross = bodyparts_cross[[0,3,4,7]]

print(uniqueID,paint)
print(bodyparts_floor, len(bodyparts_floor))
print(bodyparts_wall, len(bodyparts_wall))
print(bodyparts_cross, len(bodyparts_cross))

num_exp = 0
for i in range(num_file):
    experiment = experiments_mscore[fileloc_set[i]]
    sexes        = experiment['sex']
    for j in range(num_well):
        sex = sexes[j]

        if sex != -1:
            num_exp = num_exp+1
            
total_events     = np.zeros((3,num_exp)).astype(int)
genders_status   = -np.ones((num_exp)).astype(int)
genotypes_status = -np.ones((num_exp)).astype(int)
radius = -np.ones((num_exp))

ii = 0
for i in range(num_file):
    experiment  = experiments_mscore[fileloc_set[i]]
    sexes       = experiment['sex']
    genotypes   = experiment['genotype']
    events_type = experiment['events type']
    events      = experiment['events']
    arena_circle= experiment['arena circle']
    experiment_index = experiments_index[i]
    
    for j in range(num_well):

        sex = sexes[j]
        genotype = genotypes[j]

        if sex != -1:
            
            event_ts   = events[j]
            event_type = events_type[j]
            well_arena = arena_circle[j]

            n_event = len(event_ts)
            n_event_HB  = event_type==1
            n_event_UWE = event_type==2
            n_event_L   = event_type==3
            
            total_events[0,ii] = sum(n_event_HB)
            total_events[1,ii] = sum(n_event_UWE)
            total_events[2,ii] = sum(n_event_L)
            
            genders_status[ii] = sex
            genotypes_status[ii] = genotype
            radius[ii] = (np.mean(well_arena, axis = 0))[2]
#             print(experiment_index,videofile_set[j])
            ii += 1

single paint
Index(['head', 'center', 'tail'], dtype='object', name='bodyparts') 3
Index(['head', 'center', 'tail'], dtype='object', name='bodyparts') 3
Index(['top', 'left', 'right', 'down'], dtype='object', name='bodyparts') 4


# tracking decoder

In [4]:
np.set_printoptions(precision=2)

filter_status = '_filtered'
# filter_status = ''

well_done_rate = np.zeros(num_exp)
ii = 0
for i in range(num_file):
# for i in [1]:
# for i in range(1):
    experiment = experiments_mscore[fileloc_set[i]]
    sexes        = experiment['sex']
    genotypes   = experiment['genotype']
    experiment_type = experiment['events type']
    events = experiment['events']
    arena_circle = experiment['arena circle']
    experiment_index = experiments_index[i]
    well_coordinate = {}
    for j in range(num_well):
        
        sex = sexes[j]
        genotype = genotypes[j]
        
        if sex != -1:
            event_ts   = events[j]
            event_type = experiment_type[j]

            well_arena = arena_circle[j]
            circle = np.mean(well_arena, axis = 0)
            d_xy = [576/2, 576/2]-circle[0:2]-48
#             parameter_mm2p = (17.4625/circle[2]/2)
            parameter_mm2p = (arena_parameters['well_radius']/circle[2])
            n_event = len(event_ts)
        
            well = 'v_'+experiment_index+'_'+videofile_set[j]
    
            hfile_wall = h5_dir+'/'+well+scorer_wall+'_el'+filter_status+'.h5'
            my_file_wall = Path(hfile_wall)
            hfile_floor = h5_dir+'/'+well+scorer_floor+'_el'+filter_status+'.h5'
            my_file_floor = Path(hfile_floor)
            hfile_cross = h5_dir+'/'+well+scorer_cross+'_el'+filter_status+'.h5'
            my_file_cross = Path(hfile_cross)

            
            wall_floor = my_file_wall.is_file() and my_file_floor.is_file()
            wall_floor_cross = wall_floor and my_file_cross.is_file()
            if wall_floor_cross:
                print(well)
                idx_frames = np.arange(frame_n)
                
                Dataframe_floor = pd.read_hdf(os.path.join(hfile_floor))
                Dataframe_wall  = pd.read_hdf(os.path.join(hfile_wall))
                Dataframe_cross = pd.read_hdf(os.path.join(hfile_cross))
                
                loc_floor, conf_floor = nly.read_loc(Dataframe_floor, scorer_floor, animalID, bodyparts_floor)
                loc_wall,  conf_wall  = nly.read_loc(Dataframe_wall,  scorer_wall,  animalID, bodyparts_wall)
                loc_cross, conf_cross = nly.read_loc(Dataframe_cross, scorer_cross, animalID, bodyparts_cross)
                loc_paint, conf_paint = nly.read_loc_paint(Dataframe_floor,scorer_floor,uniqueID, paint)
                
                thr_cf = 0.95
                
                
                fly_tracker_f = nly.tracking_analysis_or_cf(loc_floor,conf_floor,thr_cf)
                fly_tracker_f_n = np.zeros((2,numBP_floor,2,frame_n))==1
                fly_tracker_f_n[:,:,0,:] = ~fly_tracker_f
                fly_tracker_f_n[:,:,1,:] = ~fly_tracker_f
                loc_floor[fly_tracker_f_n] = np.NaN
                
#                 well_tracker_f     = np.logical_and(fly_tracker_f[0,:,:],fly_tracker_f[1,:,:])
#                 well_tracker_2eyes = np.logical_and(well_tracker_f[1,:],well_tracker_f[2,:])
#                 well_tracker_head  = np.logical_and(well_tracker_2eyes, ~well_tracker_f[0,:])
#                 loc_floor_2eyes = np.mean(loc_floor[:,1:3,:,:],axis=1)
#                 fly_tracker_f[:,0,well_tracker_head] = True
#                 loc_floor[:,0,:,well_tracker_head] = loc_floor_2eyes[:,:,well_tracker_head].T
                
#                 loc_floor     = loc_floor    [:,[0,3,4],:,:]
#                 fly_tracker_f = fly_tracker_f[:,[0,3,4],:]
                
                
                
                fly_tracker_w = nly.tracking_analysis_or_cf(loc_wall, conf_wall,thr_cf)
                fly_tracker_w_n = np.zeros((2,numBP_wall,2,frame_n))==1
                fly_tracker_w_n[:,:,0,:] = ~fly_tracker_w
                fly_tracker_w_n[:,:,1,:] = ~fly_tracker_w
                loc_wall[fly_tracker_w_n]  = np.NaN
                
#                 well_tracker_w     = np.logical_and(fly_tracker_w[0,:,:],fly_tracker_w[1,:,:])
#                 well_tracker_mouth  = np.logical_and(well_tracker_w[0,:], ~well_tracker_w[1,:])
#                 fly_tracker_w[:,1, well_tracker_mouth] = True
#                 loc_wall[:,1,:,well_tracker_mouth] = loc_wall[:,0,:,well_tracker_mouth]
                
#                 loc_wall      = loc_wall     [:,[1,2,3],:,:]
#                 fly_tracker_w = fly_tracker_w[:,[1,2,3],:]
                
                
#                 print(well, np.sum(well_tracker_head), np.sum(well_tracker_mouth))
                
                
                fly_tracker_c = nly.tracking_analysis_or_cf(loc_cross,conf_cross,thr_cf)
                fly_tracker_c_n = np.zeros((2,numBP_cross,2,frame_n))==1
                fly_tracker_c_n[:,:,0,:] = ~fly_tracker_c
                fly_tracker_c_n[:,:,1,:] = ~fly_tracker_c
                loc_cross[fly_tracker_c_n]  = np.NaN

                

                
        
                loc_floor, fly_tracker_f, fly_tracker_f_ht_off = nly.filter_loc_ht(
                    loc_floor, fly_tracker_f, 0.5/parameter_mm2p) 
                loc_wall, fly_tracker_w, fly_tracker_w_ht_off = nly.filter_loc_ht(
                    loc_wall, fly_tracker_w, 0.5/parameter_mm2p)
                

                
                loc_temp          = np.zeros((2,4,2,frame_n))
                loc_temp[:,:,:,:] = np.NaN
                


                
                
                
                fly_tracker_f_goodcf = np.ones((2,3,frame_n))<0
                fly_tracker_w_goodcf = np.ones((2,3,frame_n))<0
                fly_tracker_c_goodcf = np.ones((2,3,frame_n))<0
                

                fly_tracker_temp  = np.zeros((2,4,frame_n))
                well_tracker_temp = np.zeros((4,frame_n))<0
                
                loc_temp_floor          = np.zeros((2,1,2,frame_n))
                loc_temp_floor[:,:,:,:] = np.NaN

                loc_temp_wall          = np.zeros((2,1,2,frame_n))
                loc_temp_wall[:,:,:,:] = np.NaN
                
                loc_temp_cross          = np.zeros((2,1,2,frame_n))
                loc_temp_cross[:,:,:,:] = np.NaN
                loc_temp_cross[:,0,:,:] = np.mean(loc_cross, axis=1)
                
                conf_temp_cross = np.zeros((2,1,frame_n))
                conf_temp_cross[:,0,:] = np.mean(conf_cross, axis=1)
                fly_tracker_c_bp       = conf_temp_cross[:,0,:]>=thr_cf
                

    
                
    
    
    
    
    
    
                thr_cf   = 0.99
                thr_prox = 0.5/parameter_mm2p
                well_left = np.zeros((3,frame_n))>0
                # baseline added to loc_temp, any good pair of body part detections
                ### Top View and
                for i_bp in range(3):
                    loc_temp_floor[:,0,:,:] = loc_floor[:,i_bp,:,:]
                    fly_tracker_f_bp  = fly_tracker_f[:,i_bp,:]
                    ###Top View  Side View  Cross View
                    fly_tracker_f_bp_e  = nly.tracking_analysis_eql(loc_temp_floor, 1, 1)
                    fly_tracker_f_bp_and = np.logical_and(fly_tracker_f_bp[0,:], fly_tracker_f_bp[1,:])
                    fly_tracker_f_bp_and = np.logical_and(fly_tracker_f_bp_and, fly_tracker_f_bp_e)
                    loc_temp[:,i_bp,:,fly_tracker_f_bp_and] = loc_floor[:,i_bp,:,fly_tracker_f_bp_and]
                    for i_fly in range(n_ID):
                        fly_tracker_temp[i_fly,  i_bp,fly_tracker_f_bp_and]           = 1
                    well_left[i_bp,:] = ~fly_tracker_f_bp_and # include fly_tracker_f_bp_e
                    
                ###Top View xor with Side View
                for i_bp in range(3):
                    
                    loc_temp_floor[:,0,:,:] = loc_floor[:,i_bp,:,:]
                    fly_tracker_f_bp  = fly_tracker_f[:,i_bp,:]
                    loc_temp_wall[:,0,:,:]  = loc_wall[:,i_bp,:,:]
                    fly_tracker_w_bp    = fly_tracker_w[:,i_bp,:]
                    
                    fly_tracker_f_bp_e   = nly.tracking_analysis_eql(loc_temp_floor, 1, 1)
                    fly_tracker_f_bp_xor = np.logical_xor(fly_tracker_f_bp[0,:], fly_tracker_f_bp[1,:])
                    fly_tracker_f_bp_xor = np.logical_or(fly_tracker_f_bp_xor, ~fly_tracker_f_bp_e)
                    idx_frames_f_bp_xor  = idx_frames[fly_tracker_f_bp_xor]

                    loc_temp,  fly_tracker_f_bp_w,  = tkr_dec.floor_tracker_xor_plus(
                        idx_frames_f_bp_xor, 
                        loc_temp_floor, loc_temp_wall,  loc_temp,
                        conf_wall[:,i_bp,:], 
                        i_bp, thr_cf, thr_prox)

                    for i_fly in range(n_ID):
                        fly_tracker_temp[i_fly,  i_bp,fly_tracker_f_bp_w[i_fly,:]==0] = 1
                        fly_tracker_temp[i_fly,  i_bp,fly_tracker_f_bp_w[i_fly,:]>0]  = 2


                    well_done = np.logical_or(fly_tracker_f_bp_xor, ~well_left[i_bp,:] )
                    well_left[i_bp,:] = ~well_done # include fly_tracker_f_bp_e

                ### Side View  and  
                for i_bp in range(3):
                    loc_temp_wall[:,0,:,:]  = loc_wall[:,i_bp,:,:]
                    fly_tracker_w_bp    = fly_tracker_w[:,i_bp,:]
                    ### Side View  Cross View
                    fly_tracker_w_bp_e   = nly.tracking_analysis_eql(loc_temp_wall, 1, 1)

                    fly_tracker_w_bp_and = np.logical_and(fly_tracker_w_bp[0,:], fly_tracker_w_bp[1,:])
                    fly_tracker_w_bp_and = np.logical_and(fly_tracker_w_bp_and,  fly_tracker_w_bp_e)
                    fly_tracker_w_bp_and = np.logical_and(fly_tracker_w_bp_and,  well_left[i_bp,:])
                    idx_frames_w_bp_and  = idx_frames[fly_tracker_w_bp_and]
                    loc_temp = tkr_dec.wall_tracker_and(idx_frames_w_bp_and, loc_temp_wall, loc_temp, i_bp)                
                    for i_fly in range(n_ID):
                        fly_tracker_temp[i_fly,  i_bp,fly_tracker_w_bp_and]           = 2        
                    well_done = ~ well_left[i_bp,:]
                    well_done = np.logical_or(fly_tracker_w_bp_and, well_done)
                    well_left[i_bp,:] = ~ well_done
                    
                ### Side View  xor
                thr_prox_w = 3/parameter_mm2p
                for i_bp in range(3):
                    loc_temp_wall[:,0,:,:]  = loc_wall[:,i_bp,:,:]
                    fly_tracker_w_bp    = fly_tracker_w[:,i_bp,:]
                    
                    fly_tracker_w_bp_e   = nly.tracking_analysis_eql(loc_temp_wall, 1, 1)


                    fly_tracker_w_bp_xor = np.logical_xor(fly_tracker_w_bp[0,:], fly_tracker_w_bp[1,:])
                    fly_tracker_w_bp_xor = np.logical_or (fly_tracker_w_bp_xor, ~fly_tracker_w_bp_e)
                    fly_tracker_w_bp_xor = np.logical_and(fly_tracker_w_bp_xor, well_left[i_bp,:])
                    idx_frames_w_bp_xor = idx_frames[fly_tracker_w_bp_xor]
 
                    
                    loc_temp, fly_tracker_w_bp_c = tkr_dec.wall_tracker_xor_plus(
                        idx_frames_w_bp_xor, loc_temp_wall, loc_temp, i_bp, thr_prox_w)

                    for i_fly in range(n_ID):
                        fly_tracker_temp[i_fly,  i_bp,fly_tracker_w_bp_c[i_fly,:]==0] = 2
                    well_tracker_w_bp_c   = np.logical_or(fly_tracker_w_bp_c[0,:]>=0,  fly_tracker_w_bp_c[1,:]>=0)
                    well_done = ~ well_left[i_bp,:]
                    well_done = np.logical_or(well_tracker_w_bp_c, well_done)
                    well_left[i_bp,:] = ~ well_done

                    
                ### Cross View
                well_left = np.logical_and(np.logical_and(well_left[0,:],well_left[1,:]),well_left[2,:])
                well_done = ~well_left

                fly_tracker_c_bp_e    = nly.tracking_analysis_eql(loc_temp_cross, 1, 1)

                fly_tracker_c_bp_and = np.logical_and(fly_tracker_c_bp[0,:], fly_tracker_c_bp[1,:])
                fly_tracker_c_bp_and = np.logical_and(fly_tracker_c_bp_and,  fly_tracker_c_bp_e)
                
                fly_tracker_c_bp_and_left = np.logical_and(fly_tracker_c_bp_and,  well_left)
                loc_temp[:,3,:,fly_tracker_c_bp_and_left] = loc_temp_cross[:,0,:,fly_tracker_c_bp_and_left]
                
                fly_tracker_c_bp_and_done = np.logical_and(fly_tracker_c_bp_and,  well_done)
                idx_frames_c_bp_and = idx_frames[fly_tracker_c_bp_and_done]
                
                
                loc_temp = tkr_dec.cross_tracker_and(idx_frames_c_bp_and, loc_temp_cross, loc_temp)
                
                fly_tracker_c_bp_xor = np.logical_xor(fly_tracker_c_bp[0,:], fly_tracker_c_bp[1,:])
                fly_tracker_c_bp_xor = np.logical_or(fly_tracker_c_bp_xor, ~fly_tracker_c_bp_e)
                fly_tracker_c_bp_xor = np.logical_and(fly_tracker_c_bp_xor, well_done)

                idx_frames_c = idx_frames[fly_tracker_c_bp_xor]
                

                
                loc_temp, fly_tracker_fw_bp_c = tkr_dec.cross_tracker_xor_plus(
                                           idx_frames_c, 
                                           loc_temp, loc_temp_cross, conf_temp_cross,
                                            thr_prox)

                
                for i_fly in range(n_ID):

                    fly_tracker_temp[i_fly, 3, fly_tracker_c_bp_and_left]     = 3
                    fly_tracker_temp[i_fly, 3, fly_tracker_c_bp_and_done]     = 3
                    fly_tracker_temp[i_fly, 3, fly_tracker_fw_bp_c[i_fly,:]>0]= 3


                well_done          = nly.tracking_analysis_2parts(fly_tracker_temp)
                well_done_rate[ii] = (sum(well_done)/frame_n)

                
                print(well,"%.5f" % (sum(well_done)/frame_n))
                npz_file_name = h5_dir+'/'+well+'_el'+filter_status+'.npz'
                np.savez(npz_file_name, 
                         loc_temp = loc_temp, 
                         fly_tracker_temp  = fly_tracker_temp,
                         well_tracker_temp = well_tracker_temp)
                # Access the loaded arrays
                loaded_data = np.load(npz_file_name)
                loc_temp          = loaded_data['loc_temp']
                fly_tracker_temp  = loaded_data['fly_tracker_temp']
                well_tracker_temp = loaded_data['well_tracker_temp']
                well_done          = nly.tracking_analysis_2parts(fly_tracker_temp)
                well_done_rate[ii] = (sum(well_done)/frame_n)
                print(well,"%.5f" % (sum(well_done)/frame_n))
            else:
                print('no h files')
            ii = ii+1
print(well_done_rate[0:ii])

v_05_08_24_exp1_A1
v_05_08_24_exp1_A1 0.45620
v_05_08_24_exp1_A1 0.45620
v_05_08_24_exp1_B1
v_05_08_24_exp1_B1 0.64147
v_05_08_24_exp1_B1 0.64147
v_05_08_24_exp1_D1
v_05_08_24_exp1_D1 0.41498
v_05_08_24_exp1_D1 0.41498
v_05_08_24_exp1_A2
v_05_08_24_exp1_A2 0.58268
v_05_08_24_exp1_A2 0.58268
v_05_08_24_exp1_B2
v_05_08_24_exp1_B2 0.35752
v_05_08_24_exp1_B2 0.35752
v_05_08_24_exp2_C1
v_05_08_24_exp2_C1 0.72635
v_05_08_24_exp2_C1 0.72635
v_05_08_24_exp2_D1
v_05_08_24_exp2_D1 0.82371
v_05_08_24_exp2_D1 0.82371
v_05_08_24_exp2_A2
v_05_08_24_exp2_A2 0.66160
v_05_08_24_exp2_A2 0.66160
v_05_08_24_exp2_B2
v_05_08_24_exp2_B2 0.70894
v_05_08_24_exp2_B2 0.70894
v_05_08_24_exp2_D2
v_05_08_24_exp2_D2 0.73108
v_05_08_24_exp2_D2 0.73108
v_05_09_24_exp1_A1
v_05_09_24_exp1_A1 0.66497
v_05_09_24_exp1_A1 0.66497
v_05_09_24_exp1_C1
v_05_09_24_exp1_C1 0.58529
v_05_09_24_exp1_C1 0.58529
v_05_09_24_exp1_D1
v_05_09_24_exp1_D1 0.62737
v_05_09_24_exp1_D1 0.62737
v_05_09_24_exp1_A2
v_05_09_24_exp1_A2 0.74969
v_05_

In [8]:
print(well_done_rate[0:ii],"%.3f" % np.mean(well_done_rate[0:ii]), "%.3f" % np.std(well_done_rate[0:ii]))

[0.38 0.66 0.42 0.6  0.35 0.74 0.82 0.67 0.71 0.74 0.67 0.6  0.64 0.76
 0.64 0.84 0.78 0.54 0.81 0.72 0.82 0.59 0.76 0.84 0.68 0.71 0.75 0.57
 0.54 0.65 0.7  0.81 0.61 0.72 0.76 0.71 0.61 0.75 0.59 0.6  0.62 0.73
 0.85 0.71 0.74 0.63 0.58 0.7  0.79 0.7  0.57 0.5  0.7  0.71 0.63 0.53
 0.53 0.44 0.68 0.67 0.97 0.54 0.92 0.78 0.56 0.56 0.93 0.77 0.47 0.49
 0.53 0.89 0.8  0.69 0.77 0.89 0.76 0.74 0.34 0.51 0.62 0.45 0.63 0.49
 0.35 0.75] 0.663 0.136


In [8]:
print(well_done_rate[0:ii],"%.3f" % np.mean(well_done_rate[0:ii]), "%.3f" % np.std(well_done_rate[0:ii]))

[0.95 0.93 0.74 0.94 0.81 0.74 0.94 0.78 0.89 0.92 0.86 0.91 0.72 0.9
 0.94 0.95 0.93 0.92 0.95 0.9  0.91 0.82 0.9  0.95 0.95 0.94 0.98 0.85
 0.55 0.86 0.88 0.97 0.9  0.9  0.92 0.92 0.85 0.94 0.94 0.81 0.87 0.93
 0.94 0.92 0.92 0.89 0.93 0.84 0.94 0.91 0.79 0.77 0.94 0.94 0.93 0.72
 0.8  0.74 0.84 0.92 0.98 0.8  0.97 0.89 0.95 0.79 0.97 0.93 0.76 0.81
 0.87 0.98 0.94 0.77 0.78 0.98 0.96 0.89 0.74 0.82 0.87 0.78 0.77 0.84
 0.79 0.94] 0.877 0.081


In [9]:
np.set_printoptions(precision=2)
# filter_status = '_filtered'
filter_status = '_filtered'
well_done_rate_body_center = np.zeros(num_exp)
well_done_rate_2partsOrCr = np.zeros(num_exp)
well_done_rate_2parts = np.zeros(num_exp)
well_done_rate_body_center_selected = np.zeros(num_exp)
well_done_rate_2partsOrCr_selected = np.zeros(num_exp)
well_done_rate_2parts_selected = np.zeros(num_exp)
frame_n_e = frame_rate*60*30
ii = 0
for i in range(num_file):
# for i in range(1):
    experiment = experiments_mscore[fileloc_set[i]]
    sexes        = experiment['sex']
    genotypes   = experiment['genotype']
    experiment_type = experiment['events type']
    events = experiment['events']
    arena_circle = experiment['arena circle']
    experiment_index = experiments_index[i]
    start_t = np.max(experiment['starting_time'])
    well_tracker_select = np.logical_and(np.arange(frame_n)>=start_t,np.arange(frame_n)<start_t+frame_n_e)
    print('starting_time',start_t, np.sum(well_tracker_select))
    well_coordinate = {}
    for j in range(num_well):
#     for j in range(1):
        sex = sexes[j]
        genotype = genotypes[j]
        
        if sex != -1:
            event_ts   = events[j]
            event_type = experiment_type[j]

            well_arena = arena_circle[j]
            circle = np.mean(well_arena, axis = 0)
            d_xy = [576/2, 576/2]-circle[0:2]-48
            n_event = len(event_ts)
        
            well = 'v_'+experiment_index+'_'+videofile_set[j]
            
            hfile_wall = h5_dir+'/'+well+scorer_wall+'_el'+filter_status+'.h5'
            my_file_wall = Path(hfile_wall)
            hfile_floor = h5_dir+'/'+well+scorer_floor+'_el'+filter_status+'.h5'
            my_file_floor = Path(hfile_floor)
            hfile_cross = h5_dir+'/'+well+scorer_cross+'_el'+filter_status+'.h5'
            my_file_cross = Path(hfile_cross)
            
            wall_floor = my_file_wall.is_file() and my_file_floor.is_file()
            wall_floor_cross = wall_floor and my_file_cross.is_file()
            if wall_floor_cross:
                print(well)
                
                Dataframe_floor = pd.read_hdf(os.path.join(hfile_floor))
                Dataframe_wall  = pd.read_hdf(os.path.join(hfile_wall))
                Dataframe_cross = pd.read_hdf(os.path.join(hfile_cross))
                
                loc_floor, conf_floor = nly.read_loc(Dataframe_floor, scorer_floor, animalID, bodyparts_floor)
                loc_wall,  conf_wall  = nly.read_loc(Dataframe_wall,  scorer_wall,  animalID, bodyparts_wall)
                loc_cross, conf_cross = nly.read_loc(Dataframe_cross, scorer_cross, animalID, bodyparts_cross)
                loc_paint, conf_paint = nly.read_loc_paint(Dataframe_floor,scorer_floor,uniqueID, paint)
                
                idx_frames = np.arange(frame_n)
                npz_file_name = h5_dir+'/'+well+'_el'+filter_status+'.npz'
                loaded_data = np.load(npz_file_name)
                loc_temp          = loaded_data['loc_temp']
                fly_tracker_temp  = loaded_data['fly_tracker_temp']
                well_tracker_temp = loaded_data['well_tracker_temp']
        
                

                well_tracker_body_center = nly.tracking_analysis_body_center(fly_tracker_temp)
                well_tracker_2partsOrCr  = nly.tracking_analysis_2partsOrCr_loc(loc_temp)
                well_tracker_2parts      = nly.tracking_analysis_2parts(fly_tracker_temp)
                well_tracker_body_center_selected = np.logical_and(well_tracker_body_center,well_tracker_select)
                well_tracker_2partsOrCr_selected = np.logical_and(well_tracker_2partsOrCr,well_tracker_select)
                well_tracker_2parts_selected = np.logical_and(well_tracker_2parts,well_tracker_select)
                
                well_done_rate_body_center[ii] = (sum(well_tracker_body_center)/frame_n)
                well_done_rate_2partsOrCr[ii] = (sum(well_tracker_2partsOrCr)/frame_n)
                well_done_rate_2parts[ii]     = (sum(well_tracker_2parts)/frame_n)
                well_done_rate_body_center_selected[ii] = (sum(well_tracker_body_center_selected)/frame_n_e)
                well_done_rate_2partsOrCr_selected[ii] = (sum(well_tracker_2partsOrCr_selected)/frame_n_e)
                well_done_rate_2parts_selected[ii] = (sum(well_tracker_2parts_selected)/frame_n_e)
                print(well)
                print('body center',"%.3f" % well_done_rate_body_center[ii], 
                      "%.3f" % well_done_rate_body_center_selected[ii])
                print('2 parts or Cr',"%.3f" % well_done_rate_2partsOrCr[ii],  
                      "%.3f" % well_done_rate_2partsOrCr_selected[ii])
                print('2 parts',"%.3f" % well_done_rate_2parts[ii],  
                      "%.3f" % well_done_rate_2parts_selected[ii])
#                 loc_temp_bp          = np.zeros((2,1,2,frame_n))
#                 loc_temp_bp[:,:,:,:] = np.NaN
                
#                 well_tracker_e = np.zeros(frame_n)<0
#                 for i_bp in [0]:    
#                     loc_temp_bp[:,0,:,:] = loc_temp[:,i_bp,:,:]
#                     well_tracker_e = np.logical_or(~nly.tracking_analysis_eql(loc_temp_bp, 3, 1),well_tracker_e)
                

#                 idx_frames_X = idx_frames[well_tracker]
#                 np.random.shuffle(idx_frames_X)
                
#                 video = output_fileloc +converted_folder+fileloc_set[i]+'well_'+videofile_set[j]+'_ts.mp4'
#                 cap = cv2.VideoCapture(video)
                
#                 i_idx = 0
#                 for idx_i in idx_frames_X:
#                     image = (output_fileloc + cropped_recording_folder +'/R72/Test/'+well+'_'+format(idx_i,'06d'))
#                     loc_temp_i         = loc_temp[:,:,:,idx_i]
#                     fly_tracker_temp_i = fly_tracker_temp[:,:,idx_i]
#                     tkr_dec.save_image_temp(cap, loc_temp_i, fly_tracker_temp_i, 
#                                     video, idx_i, image, d_xy, bodyparts_floor)
#                     i_idx += 1
#                     if i_idx == 10:
#                         break
                
            else:
                print('no h files')
            ii = ii+1
print("%.3f" % np.mean(well_done_rate_body_center[0:ii]),
      "%.3f" % np.std(well_done_rate_body_center[0:ii]),
      "%.3f" % np.mean(well_done_rate_2partsOrCr[0:ii]),
      "%.3f" % np.std(well_done_rate_2partsOrCr[0:ii]),
     )

starting_time 6409 108000
v_05_08_24_exp1_A1
v_05_08_24_exp1_A1
body center 0.879 0.879
2 parts or Cr 0.786 0.794
2 parts 0.384 0.372
v_05_08_24_exp1_B1
v_05_08_24_exp1_B1
body center 0.941 0.941
2 parts or Cr 0.918 0.921
2 parts 0.656 0.674
v_05_08_24_exp1_D1
v_05_08_24_exp1_D1
body center 0.769 0.753
2 parts or Cr 0.749 0.732
2 parts 0.421 0.400
v_05_08_24_exp1_A2
v_05_08_24_exp1_A2
body center 0.963 0.965
2 parts or Cr 0.948 0.951
2 parts 0.596 0.606
v_05_08_24_exp1_B2
v_05_08_24_exp1_B2
body center 0.883 0.895
2 parts or Cr 0.813 0.836
2 parts 0.348 0.353
starting_time 5088 108000
v_05_08_24_exp2_C1
v_05_08_24_exp2_C1
body center 0.894 0.880
2 parts or Cr 0.866 0.851
2 parts 0.740 0.731
v_05_08_24_exp2_D1
v_05_08_24_exp2_D1
body center 0.968 0.972
2 parts or Cr 0.961 0.965
2 parts 0.820 0.835
v_05_08_24_exp2_A2
v_05_08_24_exp2_A2
body center 0.891 0.888
2 parts or Cr 0.858 0.857
2 parts 0.671 0.674
v_05_08_24_exp2_B2
v_05_08_24_exp2_B2
body center 0.942 0.943
2 parts or Cr 0.922 0.

v_07_24_24_exp2_C2
body center 0.847 0.838
2 parts or Cr 0.826 0.815
2 parts 0.693 0.688
v_07_24_24_exp2_D2
v_07_24_24_exp2_D2
body center 0.908 0.911
2 parts or Cr 0.899 0.902
2 parts 0.770 0.779
starting_time 4230 108000
v_07_24_24_exp3_A1
v_07_24_24_exp3_A1
body center 0.992 0.995
2 parts or Cr 0.985 0.988
2 parts 0.886 0.878
v_07_24_24_exp3_B1
v_07_24_24_exp3_B1
body center 0.978 0.980
2 parts or Cr 0.963 0.964
2 parts 0.762 0.769
v_07_24_24_exp3_C1
v_07_24_24_exp3_C1
body center 0.921 0.926
2 parts or Cr 0.915 0.920
2 parts 0.744 0.743
v_07_24_24_exp3_D1
v_07_24_24_exp3_D1
body center 0.760 0.729
2 parts or Cr 0.728 0.701
2 parts 0.342 0.365
v_07_24_24_exp3_A2
v_07_24_24_exp3_A2
body center 0.864 0.859
2 parts or Cr 0.836 0.827
2 parts 0.509 0.484
v_07_24_24_exp3_B2
v_07_24_24_exp3_B2
body center 0.908 0.901
2 parts or Cr 0.894 0.886
2 parts 0.624 0.609
v_07_24_24_exp3_C2
v_07_24_24_exp3_C2
body center 0.803 0.802
2 parts or Cr 0.785 0.786
2 parts 0.446 0.466
starting_time 3136 10

In [10]:
print("%.3f" % np.mean(well_done_rate_body_center[0:ii]),
      "%.3f" % np.std(well_done_rate_body_center[0:ii]),
      "%.3f" % np.mean(well_done_rate_2partsOrCr[0:ii]),
      "%.3f" % np.std(well_done_rate_2partsOrCr[0:ii]),
      "%.3f" % np.mean(well_done_rate_2parts[0:ii]),
      "%.3f" % np.std(well_done_rate_2parts[0:ii]),
     )
print("%.3f" % np.mean(well_done_rate_body_center_selected[0:ii]),
      "%.3f" % np.std(well_done_rate_body_center_selected[0:ii]),
      "%.3f" % np.mean(well_done_rate_2partsOrCr_selected[0:ii]),
      "%.3f" % np.std(well_done_rate_2partsOrCr_selected[0:ii]),
      "%.3f" % np.mean(well_done_rate_2parts_selected[0:ii]),
      "%.3f" % np.std(well_done_rate_2parts_selected[0:ii]),
     )

0.914 0.059 0.893 0.066 0.663 0.136
0.916 0.063 0.895 0.070 0.665 0.140


In [6]:
np.set_printoptions(precision=2)

well_done_rate = np.zeros(num_exp)

ii = 0
for i in range(num_file):
# for i in range(1):
    experiment = experiments_mscore[fileloc_set[i]]
    sexes        = experiment['sex']
    genotypes   = experiment['genotype']
    experiment_type = experiment['events type']
    events = experiment['events']
    arena_circle = experiment['arena circle']
    experiment_index = experiments_index[i]
    well_coordinate = {}
    for j in range(num_well):
#     for j in range(1):
        sex = sexes[j]
        genotype = genotypes[j]
        
        if sex != -1:
            event_ts   = events[j]
            event_type = experiment_type[j]

            well_arena = arena_circle[j]
            circle = np.mean(well_arena, axis = 0)
            d_xy = [576/2, 576/2]-circle[0:2]-48
            n_event = len(event_ts)
        
            well = 'v_'+experiment_index+'_'+videofile_set[j]
            
            hfile_wall = h5_dir+'/'+well+scorer_wall+'_el_filtered.h5'
            my_file_wall = Path(hfile_wall)
            hfile_floor = h5_dir+'/'+well+scorer_floor+'_el_filtered.h5'
            my_file_floor = Path(hfile_floor)
            hfile_cross = h5_dir+'/'+well+scorer_cross+'_el_filtered.h5'
            my_file_cross = Path(hfile_cross)
            
            wall_floor = my_file_wall.is_file() and my_file_floor.is_file()
            wall_floor_cross = wall_floor and my_file_cross.is_file()
            if wall_floor_cross:
                print(well)
                
                Dataframe_floor = pd.read_hdf(os.path.join(hfile_floor))
                Dataframe_wall  = pd.read_hdf(os.path.join(hfile_wall))
                Dataframe_cross = pd.read_hdf(os.path.join(hfile_cross))
                
                loc_floor, conf_floor = nly.read_loc(Dataframe_floor, scorer_floor, animalID, bodyparts_floor)
                loc_wall,  conf_wall  = nly.read_loc(Dataframe_wall,  scorer_wall,  animalID, bodyparts_wall)
                loc_cross, conf_cross = nly.read_loc(Dataframe_cross, scorer_cross, animalID, bodyparts_cross)
                loc_paint, conf_paint = nly.read_loc_paint(Dataframe_floor,scorer_floor,uniqueID, paint)
                
                idx_frames = np.arange(frame_n)
                npz_file_name = h5_dir+'/'+well+'_el'+filter_status+'.npz'
                loaded_data = np.load(npz_file_name)
                loc_temp          = loaded_data['loc_temp']
                fly_tracker_temp  = loaded_data['fly_tracker_temp']
                well_tracker_temp = loaded_data['well_tracker_temp']
                
                fly_tracker_temp_3parts = (np.logical_or(fly_tracker_temp==1,fly_tracker_temp==2))[:,0:3,:]
                
                fly_tracker_temp_ht = np.logical_and(fly_tracker_temp_3parts[:,0,:],
                                                     fly_tracker_temp_3parts[:,2,:])
                fly_tracker_temp_ct = np.logical_and(fly_tracker_temp_3parts[:,1,:],
                                                     fly_tracker_temp_3parts[:,2,:])
                fly_tracker_temp_hc = np.logical_and(fly_tracker_temp_3parts[:,0,:],
                                                     fly_tracker_temp_3parts[:,1,:])
                

                                
#                 fly_tracker_temp_c  =  fly_tracker_temp_3parts[:,1,:]
                fly_tracker_temp_cr = (fly_tracker_temp == 3)[:,3,:]
                ##################### 6 cases##########################
                # both flies ht are tracked
                well_tracker_ht     = np.logical_and(fly_tracker_temp_ht[0,:],fly_tracker_temp_ht[1,:])
                #both flies ct are tracked
                well_tracker_ct      = np.logical_and(fly_tracker_temp_ct[0,:], fly_tracker_temp_ct[1,:])
                #both flies hc are tracked
                well_tracker_hc     = np.logical_and(fly_tracker_temp_hc[0,:],fly_tracker_temp_hc[1,:])
                #both flies cr are tracked
                well_tracker_cr     = np.logical_and(fly_tracker_temp_cr[0,:],fly_tracker_temp_cr[1,:])
                
                # one is ht, the other is ct
                well_tracker_ht0_ct1 = np.logical_and(fly_tracker_temp_ht[0,:],fly_tracker_temp_ct[1,:])
                well_tracker_ht1_ct0 = np.logical_and(fly_tracker_temp_ht[1,:],fly_tracker_temp_ct[0,:])
                # one is ct, the other is hc
                well_tracker_ct0_hc1 = np.logical_and(fly_tracker_temp_ct[0,:], fly_tracker_temp_hc[1,:])
                well_tracker_ct1_hc0 = np.logical_and(fly_tracker_temp_ct[1,:], fly_tracker_temp_hc[0,:])
                # one is ht, the other is hc
                well_tracker_ht0_hc1 = np.logical_and(fly_tracker_temp_ht[0,:],fly_tracker_temp_hc[1,:])
                well_tracker_ht1_hc0 = np.logical_and(fly_tracker_temp_ht[1,:],fly_tracker_temp_hc[0,:])
                
                # one is cr, the other is hc/ct/ht
                well_tracker_cr0_ht1 = np.logical_and(fly_tracker_temp_cr[0,:], fly_tracker_temp_ht[1,:])
                well_tracker_cr1_ht0 = np.logical_and(fly_tracker_temp_cr[1,:], fly_tracker_temp_ht[0,:])
                well_tracker_cr0_hc1 = np.logical_and(fly_tracker_temp_cr[0,:], fly_tracker_temp_hc[1,:])
                well_tracker_cr1_hc0 = np.logical_and(fly_tracker_temp_cr[1,:], fly_tracker_temp_hc[0,:])
                well_tracker_cr0_ct1 = np.logical_and(fly_tracker_temp_cr[0,:], fly_tracker_temp_ct[1,:])
                well_tracker_cr1_ct0 = np.logical_and(fly_tracker_temp_cr[1,:], fly_tracker_temp_ct[0,:])
                
                well_tracker_cr_ht_x = np.logical_or(well_tracker_cr0_ht1, well_tracker_cr1_ht0)
                well_tracker_cr_hc_x = np.logical_or(well_tracker_cr0_hc1, well_tracker_cr1_hc0)
                well_tracker_cr_ct_x = np.logical_or(well_tracker_cr0_ct1, well_tracker_cr1_ct0)
                
                well_tracker_cr_x = np.logical_or(well_tracker_cr_ht_x, well_tracker_cr_hc_x)
                well_tracker_cr_x = np.logical_or(well_tracker_cr_ct_x, well_tracker_cr_x)
                
                
                well_tracker_cr_ht_ct_hc = np.logical_or(well_tracker_cr_x, well_tracker_cr)
                # ht and ct
                well_tracker_ht_ct_e = np.logical_or(well_tracker_ht,  well_tracker_ct)
                well_tracker_ht_ct_x = np.logical_or(well_tracker_ht0_ct1, well_tracker_ht1_ct0)
                well_tracker_ht_ct   = np.logical_or(well_tracker_ht_ct_e, well_tracker_ht_ct_x)
                # ct and hc 
                well_tracker_ct_hc_e = np.logical_or(well_tracker_ct,  well_tracker_hc)
                well_tracker_ct_hc_x = np.logical_or(well_tracker_ct0_hc1,  well_tracker_ct1_hc0)
                well_tracker_ct_hc   = np.logical_or(well_tracker_ct_hc_e,  well_tracker_ct_hc_x)
                # ht and hc
                well_tracker_ht_hc_e = np.logical_or(well_tracker_ht,  well_tracker_hc)
                well_tracker_ht_hc_x = np.logical_or(well_tracker_ht0_hc1,  well_tracker_ht1_hc0)
                well_tracker_ht_hc   = np.logical_or(well_tracker_ht_hc_e,  well_tracker_ht_hc_x)
                
                
                
                
                
                well_tracker       = np.logical_or(well_tracker_ht_ct,  well_tracker_ct_hc)
                well_tracker       = np.logical_or(well_tracker_ht_hc,  well_tracker)
                well_tracker       = np.logical_or(well_tracker_cr_ht_ct_hc,  well_tracker)
                well_done_rate[ii] = (sum(well_tracker)/frame_n)
                
                print(well,"%.3f" % well_done_rate[ii],i,j)
                
#                 loc_temp_bp          = np.zeros((2,1,2,frame_n))
#                 loc_temp_bp[:,:,:,:] = np.NaN
                
#                 well_tracker_e = np.zeros(frame_n)<0
#                 for i_bp in [0]:    
#                     loc_temp_bp[:,0,:,:] = loc_temp[:,i_bp,:,:]
#                     well_tracker_e = np.logical_or(~nly.tracking_analysis_eql(loc_temp_bp, 3, 1),well_tracker_e)
                

#                 idx_frames_X = idx_frames[well_tracker]
#                 np.random.shuffle(idx_frames_X)
                
#                 video = output_fileloc +converted_folder+fileloc_set[i]+'well_'+videofile_set[j]+'_ts.mp4'
#                 cap = cv2.VideoCapture(video)
                
#                 i_idx = 0
#                 for idx_i in idx_frames_X:
#                     image = (output_fileloc + cropped_recording_folder +'/R72/Test/'+well+'_'+format(idx_i,'06d'))
#                     loc_temp_i         = loc_temp[:,:,:,idx_i]
#                     fly_tracker_temp_i = fly_tracker_temp[:,:,idx_i]
#                     tkr_dec.save_image_temp(cap, loc_temp_i, fly_tracker_temp_i, 
#                                     video, idx_i, image, d_xy, bodyparts_floor)
#                     i_idx += 1
#                     if i_idx == 10:
#                         break
                
            else:
                print('no h files')
            ii = ii+1
print(well_done_rate[0:ii],"%.3f" % np.mean(well_done_rate[0:ii]), "%.3f" % np.std(well_done_rate[0:ii]))

v_05_08_24_exp1_A1
v_05_08_24_exp1_A1 0.908 0 0
v_05_08_24_exp1_B1
v_05_08_24_exp1_B1 0.838 0 1
v_05_08_24_exp1_D1
v_05_08_24_exp1_D1 0.689 0 3
v_05_08_24_exp1_A2
v_05_08_24_exp1_A2 0.843 0 4
v_05_08_24_exp1_B2
v_05_08_24_exp1_B2 0.759 0 5
v_05_08_24_exp2_C1
v_05_08_24_exp2_C1 0.755 1 2
v_05_08_24_exp2_D1
v_05_08_24_exp2_D1 0.810 1 3
v_05_08_24_exp2_A2
v_05_08_24_exp2_A2 0.696 1 4
v_05_08_24_exp2_B2
v_05_08_24_exp2_B2 0.738 1 5
v_05_08_24_exp2_D2
v_05_08_24_exp2_D2 0.743 1 7
v_05_09_24_exp1_A1
v_05_09_24_exp1_A1 0.770 2 0
v_05_09_24_exp1_C1
v_05_09_24_exp1_C1 0.890 2 2
v_05_09_24_exp1_D1
v_05_09_24_exp1_D1 0.638 2 3
v_05_09_24_exp1_A2
v_05_09_24_exp1_A2 0.774 2 4
v_05_09_24_exp1_B2
v_05_09_24_exp1_B2 0.837 2 5
v_05_09_24_exp1_C2
v_05_09_24_exp1_C2 0.851 2 6
v_05_09_24_exp1_D2
v_05_09_24_exp1_D2 0.721 2 7
v_05_09_24_exp2_B1
v_05_09_24_exp2_B1 0.884 3 1
v_05_09_24_exp2_C1
v_05_09_24_exp2_C1 0.788 3 2
v_05_09_24_exp2_D1
v_05_09_24_exp2_D1 0.713 3 3
v_05_09_24_exp2_A2
v_05_09_24_exp2_A2 0.

In [11]:
print(well_done_rate[0:ii],"%.3f" % np.mean(well_done_rate[0:ii]), "%.3f" % np.std(well_done_rate[0:ii]))

[0.38 0.66 0.42 0.6  0.35 0.74 0.82 0.67 0.71 0.74 0.67 0.6  0.64 0.76
 0.64 0.84 0.78 0.54 0.81 0.72 0.82 0.59 0.76 0.84 0.68 0.71 0.75 0.57
 0.54 0.65 0.7  0.81 0.61 0.72 0.76 0.71 0.61 0.75 0.59 0.6  0.62 0.73
 0.85 0.71 0.74 0.63 0.58 0.7  0.79 0.7  0.57 0.5  0.7  0.71 0.63 0.53
 0.53 0.44 0.68 0.67 0.97 0.54 0.92 0.78 0.56 0.56 0.93 0.77 0.47 0.49
 0.53 0.89 0.8  0.69 0.77 0.89 0.76 0.74 0.34 0.51 0.62 0.45 0.63 0.49
 0.35 0.75] 0.663 0.136


In [ ]:
def save_image_fwc(cap, loc_floor_i, loc_wall_i, loc_cross_i, 
                   conf_f_i, conf_w_i, conf_c_i, video, idx_image, image, d_xy, bodyparts_floor):
    color_floor = [(0,0,255),(0,255,0),]
    color_wall  = [(0,255,255),(255,255,0),]
    color_cross = [(255,0,0),(255,0,255)]
    f = 0;w = 0;c = 0;
    yyyy = 0
    cap.set(cv2.CAP_PROP_POS_FRAMES, idx_image)
    res, frame = cap.read()
    for i_bp in range(3):
        for i_fly in range(n_ID):
            dot_xy = (loc_floor_i[i_fly,i_bp,:]-d_xy).astype('int')
            try:
                cv2.circle(frame, dot_xy, 2, color_floor[i_fly], i_bp+1)
                cv2.putText(frame, 'f'+ str(i_fly) + ' ' + "%.2f" %conf_f_i[i_fly,i_bp]+' '+bodyparts_floor[i_bp], 
                        (500,480+i_bp*10+i_fly*30), font, .35, color_floor[i_fly])
                f = 1
            except:
                xxxx = 1


    for i_bp in range(3):
        for i_fly in range(n_ID):
            dot_xy = (loc_wall_i[i_fly,i_bp,:]-d_xy).astype('int')
            try:
                cv2.circle(frame, dot_xy, i_bp+1, color_wall[i_fly], 1)
                cv2.putText(frame, 'w'+ str(i_fly) + ' ' + "%.2f" %conf_w_i[i_fly,i_bp]+' '+bodyparts_floor[i_bp], 
                        (500,10+i_bp*10+i_fly*30), font, .35, color_wall[i_fly])
                w = 1
            except:
                xxxx = 1


    for i_fly in range(n_ID):
        dot_xy = (loc_cross_i[i_fly,0,:]-d_xy).astype('int')
        try:
            cv2.circle(frame, dot_xy, 2, color_cross[i_fly], i_bp+1)
            cv2.putText(frame, 'c'+ str(i_fly) + ' ' +"%.2f" %conf_c_i[i_fly], 
                        (520,200+i_fly*10), font, .35, color_cross[i_fly])
            c = 1
        except:
            xxxx = 1

    cv2.imwrite(image + '_' +str(f)+str(w)+str(c)+'_rec.png', frame, [cv2.IMWRITE_PNG_COMPRESSION, 0])
        
def save_image_temp(cap, loc_i, fly_tracker_temp_i, video, idx_image, image, d_xy, bodyparts_floor):
    #             red       green     blue      white
    color_temp = ((0,0,255),(0,255,0),(255,0,0),(255,255,255))
    tracker    = ['n','f','w','cr']
    bodyparts  = ['h','c','t','cr']
    yyyy = 0
    
    cap.set(cv2.CAP_PROP_POS_FRAMES, idx_image)
    res, frame = cap.read()
    for i_bp in range(4):
        for i_fly in range(n_ID):
            tracker_id = (fly_tracker_temp_i[i_fly,i_bp]).astype(int)
            dot_xy = (loc_i[i_fly,i_bp,:]-d_xy).astype('int')
            try:
                cv2.circle(frame, dot_xy, (i_bp+1)+(i_fly*2), color_temp[tracker_id], 1)
                cv2.circle(frame, (120,5+i_bp*15+i_fly*70), (i_bp+1)+(i_fly*2), color_temp[tracker_id], 1)
                cv2.putText(frame, 
                            tracker[tracker_id]+' '+str(tracker_id)+' '+str(i_fly)+' '+bodyparts[i_bp]+' '+str(dot_xy[0])+' '+str(dot_xy[1]),
                            (5, 10+i_bp*15+i_fly*70), font, .35, color_temp[tracker_id])
                yyyy = 1
            except:
                xxxx = 1

    if yyyy == 1:
        cv2.imwrite(image + '_temp_rec.png', frame, [cv2.IMWRITE_PNG_COMPRESSION, 0])
#     else:
#         cv2.imwrite(image + '_temp_rec_No_tracking.png', frame, [cv2.IMWRITE_PNG_COMPRESSION, 0])

In [ ]:
np.set_printoptions(precision=2)
filter_status = '_filtered'

l_thr = 300
dpixel = 1
d_thr_l = np.arange(l_thr)*dpixel
d_thr_l_up_limit = np.arange(l_thr)*dpixel+3

twoflies    = np.zeros((l_thr,num_exp,3))
d_max_exp  = -np.ones((frame_n,num_exp))
bp_max_exp = -np.ones((2,frame_n,num_exp))
well_done_rate = np.zeros(num_exp)
ii = 0

for i in [0]:
    experiment = experiments_mscore[fileloc_set[i]]
    sexes        = experiment['sex']
    genotypes   = experiment['genotype']
    experiment_type = experiment['events type']
    events = experiment['events']
    arena_circle = experiment['arena circle']
    experiment_index = experiments_index[i]
    well_coordinate = {}

    for j in [1]:
        sex = sexes[j]
        genotype = genotypes[j]
        
        if sex != -1:
            event_ts   = events[j]
            event_type = experiment_type[j]

            well_arena = arena_circle[j]
            circle = np.mean(well_arena, axis = 0)
            d_xy = [576/2, 576/2]-circle[0:2]-48
            parameter_mm2p = (17.4625/circle[2]/2)
            n_event = len(event_ts)
        
            well = 'v_'+experiment_index+'_'+videofile_set[j]
            
            hfile_wall = h5_dir+'/'+well+scorer_wall+'_el_filtered.h5'
            my_file_wall = Path(hfile_wall)
            hfile_floor = h5_dir+'/'+well+scorer_floor+'_el_filtered.h5'
            my_file_floor = Path(hfile_floor)
            hfile_cross = h5_dir+'/'+well+scorer_cross+'_el_filtered.h5'
            my_file_cross = Path(hfile_cross)
            
            wall_floor = my_file_wall.is_file() and my_file_floor.is_file()
            wall_floor_cross = wall_floor and my_file_cross.is_file()
            if wall_floor_cross:

                idx_frames = np.arange(frame_n)
                
                Dataframe_floor = pd.read_hdf(os.path.join(hfile_floor))
                Dataframe_wall  = pd.read_hdf(os.path.join(hfile_wall))
                Dataframe_cross = pd.read_hdf(os.path.join(hfile_cross))
                
                loc_floor, conf_floor = nly.read_loc(Dataframe_floor, scorer_floor, animalID, bodyparts_floor)
                loc_wall,  conf_wall  = nly.read_loc(Dataframe_wall,  scorer_wall,  animalID, bodyparts_wall)
                loc_cross, conf_cross = nly.read_loc(Dataframe_cross, scorer_cross, animalID, bodyparts_cross)
                loc_paint, conf_paint = nly.read_loc_paint(Dataframe_floor,scorer_floor,uniqueID, paint)
                loc_temp_cross          = np.zeros((2,1,2,frame_n))
                loc_temp_cross[:,:,:,:] = np.NaN
                loc_temp_cross[:,0,:,:] = np.nanmean(loc_cross, axis=1)
                conf_temp_cross = np.zeros((2,1,frame_n))
                conf_temp_cross[:,0,:] = np.nanmean(conf_cross, axis=1)
                idx_frames = np.arange(frame_n)
                npz_file_name = h5_dir+'/'+well+'_el'+filter_status+'.npz'
                # Access the loaded arrays
                loaded_data = np.load(npz_file_name)
                loc_temp          = loaded_data['loc_temp']
                fly_tracker_temp  = loaded_data['fly_tracker_temp']
                well_tracker_temp = loaded_data['well_tracker_temp']
                
                video = output_fileloc +converted_folder+fileloc_set[i]+'well_'+videofile_set[j]+'_ts.mp4'
                cap = cv2.VideoCapture(video)

                idx_i = 10905
                print(idx_i)
                image = (output_fileloc + cropped_recording_folder +'/'+well+'_'+format(idx_i,'06d'))
                loc_temp_i         = loc_temp[:,:,:,idx_i]
                fly_tracker_temp_i = fly_tracker_temp[:,:,idx_i]
                loc_f_i = loc_floor[:,:,:,idx_i]
                loc_w_i = loc_wall[:,:,:,idx_i]
                loc_c_i = loc_temp_cross[:,:,:,idx_i]

                cf_f_i = conf_floor[:,:,idx_i]
                cf_w_i = conf_wall [:,:,idx_i]
                cf_c_i = conf_temp_cross[:,0,idx_i]

                tkr_dec.save_image_fwc(cap, loc_f_i, loc_w_i, loc_c_i, 
                                     cf_f_i,  cf_w_i,  cf_c_i, video, idx_i, image, d_xy, bodyparts_floor)
                    
                loc_temp_i         = loc_temp[:,:,:,idx_i]
                fly_tracker_temp_i = fly_tracker_temp[:,:,idx_i]
                tkr_dec.save_image_temp(cap, loc_temp_i, fly_tracker_temp_i, 
                                video, idx_i, image, d_xy, bodyparts_floor)
                print(well, i,j)
                ############################################################
                thr_cf = 0.95
                
                fly_tracker_f = nly.tracking_analysis_or_cf(loc_floor,conf_floor,thr_cf)
                fly_tracker_f_n = np.zeros((2,3,2,frame_n))==1
                fly_tracker_f_n[:,:,0,:] = ~fly_tracker_f
                fly_tracker_f_n[:,:,1,:] = ~fly_tracker_f
                loc_floor[fly_tracker_f_n] = np.NaN
                
                fly_tracker_w = nly.tracking_analysis_or_cf(loc_wall, conf_wall,thr_cf)
                fly_tracker_w_n = np.zeros((2,3,2,frame_n))==1
                fly_tracker_w_n[:,:,0,:] = ~fly_tracker_w
                fly_tracker_w_n[:,:,1,:] = ~fly_tracker_w
                loc_wall[fly_tracker_w_n]  = np.NaN
                
                fly_tracker_c = nly.tracking_analysis_or_cf(loc_cross,conf_cross,thr_cf)
                fly_tracker_c_n = np.zeros((2,4,2,frame_n))==1
                fly_tracker_c_n[:,:,0,:] = ~fly_tracker_c
                fly_tracker_c_n[:,:,1,:] = ~fly_tracker_c
                loc_cross[fly_tracker_c_n]  = np.NaN
                
                
                fly_tracker_f_goodcf = np.ones((2,3,frame_n))<0
                fly_tracker_w_goodcf = np.ones((2,3,frame_n))<0
                fly_tracker_c_goodcf = np.ones((2,3,frame_n))<0
                

                
                loc_temp          = np.zeros((2,4,2,frame_n))
                loc_temp[:,:,:,:] = np.NaN

                fly_tracker_temp  = np.zeros((2,4,frame_n))
                well_tracker_temp = np.zeros((4,frame_n))<0
                
                loc_temp_floor          = np.zeros((2,1,2,frame_n))
                loc_temp_floor[:,:,:,:] = np.NaN

                
                loc_temp_wall          = np.zeros((2,1,2,frame_n))
                loc_temp_wall[:,:,:,:] = np.NaN
                
                loc_temp_cross          = np.zeros((2,1,2,frame_n))
                loc_temp_cross[:,:,:,:] = np.NaN
                loc_temp_cross[:,0,:,:] = np.mean(loc_cross, axis=1)
                
                conf_temp_cross = np.zeros((2,1,frame_n))
                conf_temp_cross[:,0,:] = np.mean(conf_cross, axis=1)
                fly_tracker_c_bp = conf_temp_cross[:,0,:]>=thr_cf
                
                loc_floor_m          = np.zeros((2,1,2,frame_n))
                loc_floor_m[:,:,:,:] = np.NaN
                loc_floor_m[:,0,:,:] = np.nanmean(loc_floor, axis=1)
                
                loc_wall_m          = np.zeros((2,1,2,frame_n))
                loc_wall_m[:,:,:,:] = np.NaN
                loc_wall_m[:,0,:,:] = np.nanmean(loc_wall, axis=1)
                

                
                thr_cf   = 0.95
                thr_prox = 20
                
                well_left = np.zeros((3,frame_n))>0
                for i_bp in range(3):
                    
                    loc_temp_floor[:,0,:,:] = loc_floor[:,i_bp,:,:]
                    fly_tracker_f_bp  = fly_tracker_f[:,i_bp,:]
                    loc_temp_wall[:,0,:,:]  = loc_wall[:,i_bp,:,:]
                    fly_tracker_w_bp    = fly_tracker_w[:,i_bp,:]
                    
                    ###Top View  Side View  Cross View
                    fly_tracker_f_bp_e  = nly.tracking_analysis_eql(loc_temp_floor, 1, 1)
                    fly_tracker_f_bp_and = np.logical_and(fly_tracker_f_bp[0,:], fly_tracker_f_bp[1,:])
                    fly_tracker_f_bp_and = np.logical_and(fly_tracker_f_bp_and, fly_tracker_f_bp_e)
                    fly_tracker_f_bp_xor = np.logical_xor(fly_tracker_f_bp[0,:], fly_tracker_f_bp[1,:])
                    fly_tracker_f_bp_xor = np.logical_or(fly_tracker_f_bp_xor, ~fly_tracker_f_bp_e)
                    idx_frames_f_bp_xor = idx_frames[fly_tracker_f_bp_xor]


                    loc_temp,  fly_tracker_f_bp_w,  = tkr_dec.floor_tracker_xor_plus(
                        idx_frames_f_bp_xor, 
                        loc_temp_floor, loc_temp_wall,  loc_temp,
                        conf_wall[:,i_bp,:], 
                        i_bp, thr_cf, thr_prox)

                    loc_temp[:,i_bp,:,fly_tracker_f_bp_and] = loc_floor[:,i_bp,:,fly_tracker_f_bp_and]
                    
                    for i_fly in range(n_ID):
                        fly_tracker_temp[i_fly,  i_bp,fly_tracker_f_bp_and]           = 1
                        fly_tracker_temp[i_fly,  i_bp,fly_tracker_f_bp_w[i_fly,:]==0] = 1
                        fly_tracker_temp[i_fly,  i_bp,fly_tracker_f_bp_w[i_fly,:]>0]  = 2


                    well_tracker_f_bp_w   = np.logical_or(fly_tracker_f_bp_w[0,:]>=0,  fly_tracker_f_bp_w[1,:]>=0)

                    well_done = np.logical_or(fly_tracker_f_bp_xor, fly_tracker_f_bp_and)
                    well_left[i_bp,:] = ~well_done # include fly_tracker_f_bp_e
                    
                thr_prox_w = 3.5/parameter_mm2p
                for i_bp in range(3): 
                    loc_temp_wall[:,0,:,:]  = loc_wall[:,i_bp,:,:]
                    fly_tracker_w_bp    = fly_tracker_w[:,i_bp,:]
                    ### Side View  Cross View
                    fly_tracker_w_bp_e   = nly.tracking_analysis_eql(loc_temp_wall, 1, 1)

                    fly_tracker_w_bp_and = np.logical_and(fly_tracker_w_bp[0,:], fly_tracker_w_bp[1,:])
                    fly_tracker_w_bp_and = np.logical_and(fly_tracker_w_bp_and,  fly_tracker_w_bp_e)
                    fly_tracker_w_bp_and = np.logical_and(fly_tracker_w_bp_and,  well_left[i_bp,:])

                    fly_tracker_w_bp_xor = np.logical_xor(fly_tracker_w_bp[0,:], fly_tracker_w_bp[1,:])
                    fly_tracker_w_bp_xor = np.logical_or(fly_tracker_w_bp_xor, ~fly_tracker_w_bp_e)
                    fly_tracker_w_bp_xor = np.logical_and(fly_tracker_w_bp_xor, well_left[i_bp,:])

                    idx_frames_w_bp_xor = idx_frames[fly_tracker_w_bp_xor]
                    idx_frames_w_bp_and = idx_frames[fly_tracker_w_bp_and]
                    
                    print('wall',"before",i_bp)
                    print(loc_temp[0,0,:,idx_i],fly_tracker_temp[0,0,idx_i])
                    print(loc_temp[1,0,:,idx_i],fly_tracker_temp[1,0,idx_i])
                    print('')
                    
                    loc_temp, fly_tracker_w_bp_c = tkr_dec.wall_tracker_xor_plus(
                        idx_frames_w_bp_xor, loc_temp_wall, loc_temp, i_bp, thr_prox_w)
                    
                    print('wall',"after",i_bp)
                    print(loc_temp[0,0,:,idx_i],fly_tracker_temp[0,0,idx_i])
                    print(loc_temp[1,0,:,idx_i],fly_tracker_temp[1,0,idx_i])
                    print('')
                    loc_temp = tkr_dec.wall_tracker_and(idx_frames_w_bp_and, loc_temp_wall, loc_temp,
                                                        loc_floor_m, i_bp)
                    
                    for i_fly in range(n_ID):
                        fly_tracker_temp[i_fly,  i_bp,fly_tracker_w_bp_and]           = 2
                        fly_tracker_temp[i_fly,  i_bp,fly_tracker_w_bp_c[i_fly,:]==0] = 2

                    
                    
                    well_tracker_w_bp_c   = np.logical_or(fly_tracker_w_bp_c[0,:]>=0,  fly_tracker_w_bp_c[1,:]>=0)

                    well_done_wc = np.logical_or(well_tracker_w_bp_c, fly_tracker_w_bp_and)
                    well_done = ~ well_left[i_bp,:]
                    well_done = np.logical_or(well_done_wc, well_done)
                    well_left[i_bp,:] = ~ well_done

                    
                ### Cross View
                well_left = np.logical_and(np.logical_and(well_left[0,:],well_left[1,:]),well_left[2,:])
                well_done = ~well_left
                
                fly_tracker_c_bp_e    = nly.tracking_analysis_eql(loc_temp_cross, 1, 1)

                fly_tracker_c_bp_and = np.logical_and(fly_tracker_c_bp[0,:], fly_tracker_c_bp[1,:])
                fly_tracker_c_bp_and = np.logical_and(fly_tracker_c_bp_and,  fly_tracker_c_bp_e)
                
                fly_tracker_c_bp_and_left = np.logical_and(fly_tracker_c_bp_and,  well_left)
                loc_temp[:,3,:,fly_tracker_c_bp_and_left] = loc_temp_cross[:,0,:,fly_tracker_c_bp_and_left]
                
                fly_tracker_c_bp_and_done = np.logical_and(fly_tracker_c_bp_and,  well_done)
                idx_frames_c_bp_and = idx_frames[fly_tracker_c_bp_and_done]
                
                
                loc_temp = tkr_dec.cross_tracker_and(idx_frames_c_bp_and, loc_temp_cross, loc_temp)
                
                fly_tracker_c_bp_xor = np.logical_xor(fly_tracker_c_bp[0,:], fly_tracker_c_bp[1,:])
                fly_tracker_c_bp_xor = np.logical_or(fly_tracker_c_bp_xor, ~fly_tracker_c_bp_e)
                fly_tracker_c_bp_xor = np.logical_and(fly_tracker_c_bp_xor, well_done)

                idx_frames_c = idx_frames[fly_tracker_c_bp_xor]
                

                
#                 loc_temp, fly_tracker_fw_bp_c = tkr_dec.cross_tracker_xor_plus(
#                                            idx_frames_c, 
#                                            loc_temp, loc_temp_cross, conf_temp_cross)
                print('wall')
                print(loc_temp[0,0,:,idx_i],loc_temp[1,0,:,idx_i])
                print('')
                loc_temp, fly_tracker_fw_bp_c = tkr_dec.cross_tracker_xor_plus(
                                           idx_frames_c, 
                                           loc_temp, loc_temp_cross, conf_temp_cross, thr_prox)
                for i_fly in range(n_ID):

                    fly_tracker_temp[i_fly, 3, fly_tracker_c_bp_and_left]     = 3
                    fly_tracker_temp[i_fly, 3, fly_tracker_c_bp_and_done]     = 3
                    fly_tracker_temp[i_fly, 3, fly_tracker_fw_bp_c[i_fly,:]>0]= 3
                    
                fly_tracker_temp_3parts = (np.logical_or(fly_tracker_temp==1,fly_tracker_temp==2))[:,0:3,:]
                
                fly_tracker_temp_ht = np.logical_and(fly_tracker_temp_3parts[:,0,:],
                                                     fly_tracker_temp_3parts[:,2,:])
                
                fly_tracker_temp_hct = np.logical_and(fly_tracker_temp_ht,
                                                     fly_tracker_temp_3parts[:,1,:])
                
                well_done = np.logical_and(fly_tracker_temp_hct[0,:], fly_tracker_temp_hct[1,:])

                well_done_rate[ii]     = (sum(well_done)/frame_n)
                
                

                
                

                print('wall')
                print(loc_temp[0,0,:,idx_i],loc_temp[1,0,:,idx_i])
                print('')
            else:
                print('no h files')
            ii = ii+1


In [ ]:
def cross_tracker_xor_plus(idx_frames_w_bp_e, loc_temp, loc_temp_cross, conf_temp_cross, thr_prox):
    
    frame_n = loc_temp_cross.shape[-1]
    l_bp_e = len(idx_frames_w_bp_e)
    well_tracker_f_miss = np.zeros(frame_n) <0
    
    d_1_tc = -np.ones(l_bp_e)
    d_2_tc = -np.ones(l_bp_e)

    fly_tracker_fw_bp_c  = -np.ones((2,frame_n))

    i_bp_e = 0
    i_idx_miss = 0
    i_idx_miss_why = 0

    for idx_i in idx_frames_w_bp_e:

        if np.isnan(loc_temp_cross[0,0,0,idx_i]):
            loc_i = loc_temp_cross[1,0,:,idx_i]
            fly_id_cross = 1
        else:
            loc_i = loc_temp_cross[0,0,:,idx_i]
            fly_id_cross = 0


            
        loc_temp_i = np.nanmean(loc_temp[:,0:3,:,idx_i],axis=1)
        
        d_1_tc[i_bp_e] = np.sqrt(np.sum(np.square(loc_temp_i[0,:]-loc_i)))
        d_2_tc[i_bp_e] = np.sqrt(np.sum(np.square(loc_temp_i[1,:]-loc_i)))
#         if idx_i == 55048:
#             print(d_1_tc[i_bp_e],d_2_tc[i_bp_e])
#             print(loc_temp_i, loc_i)
        if np.isnan(d_1_tc[i_bp_e]) or np.isnan(d_2_tc[i_bp_e]):
            if np.isnan(d_2_tc[i_bp_e]):
                if d_1_tc[i_bp_e] > thr_prox:
                    loc_temp[1, 3, :, idx_i] = loc_i
                    fly_tracker_fw_bp_c[1,idx_i] = fly_id_cross+1
                else:
                    loc_temp[0, 3, :, idx_i] = loc_i
                    fly_tracker_fw_bp_c[0,idx_i] = fly_id_cross+1
            else:
                if d_2_tc[i_bp_e] > thr_prox:
                    loc_temp[0, 3, :, idx_i] = loc_i
                    fly_tracker_fw_bp_c[0,idx_i] = fly_id_cross+1
                else:
                    loc_temp[1, 3, :, idx_i] = loc_i
                    fly_tracker_fw_bp_c[1,idx_i] = fly_id_cross+1
        else:
            if d_1_tc[i_bp_e] < d_2_tc[i_bp_e]:

                loc_temp[0, 3, :, idx_i] = loc_i
                fly_tracker_fw_bp_c[0,idx_i] = fly_id_cross+1
            else:
                loc_temp[1, 3, :, idx_i] = loc_i
                fly_tracker_fw_bp_c[1,idx_i] = fly_id_cross+1


        i_bp_e += 1
    return loc_temp, fly_tracker_fw_bp_c

# Undecided

In [ ]:
np.set_printoptions(precision=2)


l_thr = 100
dpixel = 1
d_thr_l = np.arange(l_thr)*dpixel
d_thr_l_up_limit = np.arange(l_thr)*dpixel+3

twoflies    = np.zeros((l_thr,num_exp,3,3))
# d_max_exp  = -np.ones((frame_n,num_exp))
# bp_max_exp = -np.ones((2,frame_n,num_exp))

ii = 0
# for i in range(num_file):
# for i in [1]:
for i in range(1):
    experiment = experiments_mscore[fileloc_set[i]]
    sexes        = experiment['sex']
    genotypes   = experiment['genotype']
    experiment_type = experiment['events type']
    events = experiment['events']
    arena_circle = experiment['arena circle']
    experiment_index = experiments_index[i]
    well_coordinate = {}
#     for j in range(num_well):
    for j in range(1):
#     for i in [7]:
        sex = sexes[j]
        genotype = genotypes[j]
        
        if sex != -1:
            event_ts   = events[j]
            event_type = experiment_type[j]

            well_arena = arena_circle[j]
            circle = np.mean(well_arena, axis = 0)
            d_xy = [576/2, 576/2]-circle[0:2]-48

            n_event = len(event_ts)
        
            well = 'v_'+experiment_index+'_'+videofile_set[j]
            
            hfile_wall = h5_dir+'/'+well+scorer_wall+'_el_filtered.h5'
            my_file_wall = Path(hfile_wall)
            hfile_floor = h5_dir+'/'+well+scorer_floor+'_el_filtered.h5'
            my_file_floor = Path(hfile_floor)
            hfile_cross = h5_dir+'/'+well+scorer_cross+'_el_filtered.h5'
            my_file_cross = Path(hfile_cross)

#             hfile_wall = h5_dir+'/'+well+scorer_wall+'_el.h5'
#             my_file_wall = Path(hfile_wall)
#             hfile_floor = h5_dir+'/'+well+scorer_floor+'_el.h5'
#             my_file_floor = Path(hfile_floor)
#             hfile_cross = h5_dir+'/'+well+scorer_cross+'_el.h5'
#             my_file_cross = Path(hfile_cross)
            
            wall_floor = my_file_wall.is_file() and my_file_floor.is_file()
            wall_floor_cross = wall_floor and my_file_cross.is_file()
            if wall_floor_cross:
                print(well)
                video = output_fileloc +converted_folder+fileloc_set[i]+'well_'+videofile_set[j]+'_ts.mp4'
                cap = cv2.VideoCapture(video)
                idx_frames = np.arange(frame_n)
                
                Dataframe_floor = pd.read_hdf(os.path.join(hfile_floor))
                Dataframe_wall  = pd.read_hdf(os.path.join(hfile_wall))
                Dataframe_cross = pd.read_hdf(os.path.join(hfile_cross))
                
                loc_floor, conf_floor = nly.read_loc(Dataframe_floor, scorer_floor, animalID, bodyparts_floor)
                loc_wall,  conf_wall  = nly.read_loc(Dataframe_wall,  scorer_wall,  animalID, bodyparts_wall)
                loc_cross, conf_cross = nly.read_loc(Dataframe_cross, scorer_cross, animalID, bodyparts_cross)
                loc_paint, conf_paint = nly.read_loc_paint(Dataframe_floor,scorer_floor,uniqueID, paint)
                

                thr_cf = 0.95


                fly_tracker_f = nly.tracking_analysis_or_cf(loc_floor,conf_floor,thr_cf)
                fly_tracker_f_n = np.zeros((2,3,2,frame_n))==1
                fly_tracker_f_n[:,:,0,:] = ~fly_tracker_f
                fly_tracker_f_n[:,:,1,:] = ~fly_tracker_f
                loc_floor[fly_tracker_f_n] = np.NaN
                
                fly_tracker_w = nly.tracking_analysis_or_cf(loc_wall, conf_wall,thr_cf)
                fly_tracker_w_n = np.zeros((2,3,2,frame_n))==1
                fly_tracker_w_n[:,:,0,:] = ~fly_tracker_w
                fly_tracker_w_n[:,:,1,:] = ~fly_tracker_w
                loc_wall[fly_tracker_w_n]  = np.NaN

                fly_tracker_c = nly.tracking_analysis_or_cf(loc_cross,conf_cross,thr_cf)
                fly_tracker_c_n = np.zeros((2,4,2,frame_n))==1
                fly_tracker_c_n[:,:,0,:] = ~fly_tracker_c
                fly_tracker_c_n[:,:,1,:] = ~fly_tracker_c
                loc_cross[fly_tracker_c_n]  = np.NaN
                loc_temp_cross          = np.zeros((2,1,2,frame_n))
                loc_temp_cross[:,0,:,:] = np.nanmean(loc_cross, axis=1)
                conf_temp_cross = np.zeros((2,1,frame_n))
                conf_temp_cross[:,0,:] = np.nanmean(conf_cross, axis=1)
                loc_temp          = np.zeros((2,4,2,frame_n))
                loc_temp[:,:,:,:] = np.NaN
                
                fly_tracker_f_e = ~nly.tracking_analysis_eql(loc_floor, 1, 2)
                idx_frames_e = idx_frames[fly_tracker_f_e]
                np.random.shuffle(idx_frames_e)
                
                jj = 0
                for idx_i in idx_frames_e:
                    loc_i_f = np.nanmean(loc_floor[:,:,:,idx_i],axis=1)
                    loc_i_w = np.nanmean(loc_wall[:,:,:,(idx_i)],axis=1)
                    loc_i_c = np.nanmean(loc_cross[:,:,:,(idx_i)],axis=1)
                    
                    d_i_fw  = np.sqrt(np.sum(np.square(loc_i_w-loc_i_f),axis=1))
                    d_i_fc  = np.sqrt(np.sum(np.square(loc_i_c-loc_i_f),axis=1))
                    
                    d_i_f = np.ones(4)
                    d_i_f[0:2] = d_i_fw
                    d_i_f[2:4] = d_i_fc
                    d_i_f[np.isnan(d_i_f)] = -1
                    idx_max = np.argmax(d_i_f)
                    if d_i_f[idx_max] > 0 and d_i_f[idx_max] > 50:
                        if idx_max==0:
                            pair = 'f_w1'
                        elif idx_max==1:
                            pair = 'f_w2'
                        elif idx_max==2:
                            pair = 'f_c1'
                        elif idx_max==3:
                            pair = 'f_c2'
                        
                        loc_f_i = loc_floor[:,:,:,idx_i]
                        loc_w_i = loc_wall[:,:,:,idx_i]
                        loc_c_i = loc_temp_cross[:,:,:,idx_i]

                        cf_f_i = conf_floor[:,:,idx_i]
                        cf_w_i = conf_wall [:,:,idx_i]
                        cf_c_i = conf_temp_cross[:,0,idx_i]
                        image = (output_fileloc + cropped_recording_folder +'/R72/Test/'+well+'_'+format(idx_i,'06d')+'_'+pair)
#                         tkr_dec.save_image_fwc(cap, loc_f_i, loc_w_i, loc_c_i, 
#                                              cf_f_i,  cf_w_i,  cf_c_i, 
#                                        video, idx_i, image, d_xy, bodyparts_floor)
                    
                    if jj==5:
                        break
                    jj += 1
                    
                    
                fly_tracker_w_e = ~nly.tracking_analysis_eql(loc_wall, 1, 2)
                idx_frames_e = idx_frames[fly_tracker_w_e]
                np.random.shuffle(idx_frames_e)
                
                jj = 0
                for idx_i in idx_frames_e:
                   
                    loc_i_f = np.nanmean(loc_floor[:,:,:,idx_i],axis=1)
                    loc_i_w = np.nanmean(loc_wall[:,:,:,(idx_i)],axis=1)
                    loc_i_c = np.nanmean(loc_cross[:,:,:,(idx_i)],axis=1)
                    
                    d_i_wf  = np.sqrt(np.sum(np.square(loc_i_w-loc_i_f),axis=1))
                    d_i_wc  = np.sqrt(np.sum(np.square(loc_i_w-loc_i_c),axis=1))
                    
                    d_i_w = np.ones(4)
                    d_i_w[0:2] = d_i_wf
                    d_i_w[2:4] = d_i_wc
                    d_i_w[np.isnan(d_i_w)] = -1
                    idx_max = np.argmax(d_i_w)
                    if d_i_w[idx_max] >= 0 and d_i_w[idx_max] > 50:
                        if idx_max==0:
                            pair = 'w_f1'
                        elif idx_max==1:
                            pair = 'w_f2'
                        elif idx_max==2:
                            pair = 'w_c1'
                        elif idx_max==3:
                            pair = 'w_c2'
                        
                        loc_f_i = loc_floor[:,:,:,idx_i]
                        loc_w_i = loc_wall[:,:,:,idx_i]
                        loc_c_i = loc_temp_cross[:,:,:,idx_i]

                        cf_f_i = conf_floor[:,:,idx_i]
                        cf_w_i = conf_wall [:,:,idx_i]
                        cf_c_i = conf_temp_cross[:,0,idx_i]
                        image = (output_fileloc + cropped_recording_folder +'/R72/Test/'+well+'_'+format(idx_i,'06d')+'_'+pair)
#                         tkr_dec.save_image_fwc(cap, loc_f_i, loc_w_i, loc_c_i, 
#                                              cf_f_i,  cf_w_i,  cf_c_i, 
#                                        video, idx_i, image, d_xy, bodyparts_floor)
                    
                    if jj == 5:
                        break
                    jj += 1
            else:
                print('no h files')
            ii = ii+1

In [ ]:
np.set_printoptions(precision=2)


l_thr = 100
dpixel = 1
d_thr_l = np.arange(l_thr)*dpixel
d_thr_l_up_limit = np.arange(l_thr)*dpixel+3

twoflies    = np.zeros((l_thr,num_exp,3,3))
# d_max_exp  = -np.ones((frame_n,num_exp))
# bp_max_exp = -np.ones((2,frame_n,num_exp))

ii = 0
for i in range(num_file):
# for i in [1]:
# for i in range(1):
    experiment = experiments_mscore[fileloc_set[i]]
    sexes        = experiment['sex']
    genotypes   = experiment['genotype']
    experiment_type = experiment['events type']
    events = experiment['events']
    arena_circle = experiment['arena circle']
    experiment_index = experiments_index[i]
    well_coordinate = {}
    for j in range(num_well):
#     for j in range(1):
#     for i in [7]:
        sex = sexes[j]
        genotype = genotypes[j]
        
        if sex != -1:
            event_ts   = events[j]
            event_type = experiment_type[j]

            well_arena = arena_circle[j]
            circle = np.mean(well_arena, axis = 0)
            d_xy = [576/2, 576/2]-circle[0:2]-48
            

            
            parameter_mm2p = (17.4625/(np.mean(well_arena, axis = 0))[2]/2)
            
            n_event = len(event_ts)
        
            well = 'v_'+experiment_index+'_'+videofile_set[j]
            
            hfile_wall = h5_dir+'/'+well+scorer_wall+'_el_filtered.h5'
            my_file_wall = Path(hfile_wall)
            hfile_floor = h5_dir+'/'+well+scorer_floor+'_el_filtered.h5'
            my_file_floor = Path(hfile_floor)
            hfile_cross = h5_dir+'/'+well+scorer_cross+'_el_filtered.h5'
            my_file_cross = Path(hfile_cross)

            hfile_wall = h5_dir+'/'+well+scorer_wall+'_el.h5'
            my_file_wall = Path(hfile_wall)
            hfile_floor = h5_dir+'/'+well+scorer_floor+'_el.h5'
            my_file_floor = Path(hfile_floor)
            hfile_cross = h5_dir+'/'+well+scorer_cross+'_el.h5'
            my_file_cross = Path(hfile_cross)
            
            wall_floor = my_file_wall.is_file() and my_file_floor.is_file()
            wall_floor_cross = wall_floor and my_file_cross.is_file()
            if wall_floor_cross:
                
                video = output_fileloc +converted_folder+fileloc_set[i]+'well_'+videofile_set[j]+'_ts.mp4'
                cap = cv2.VideoCapture(video)
                idx_frames = np.arange(frame_n)
                
                Dataframe_floor = pd.read_hdf(os.path.join(hfile_floor))
                Dataframe_wall  = pd.read_hdf(os.path.join(hfile_wall))
                Dataframe_cross = pd.read_hdf(os.path.join(hfile_cross))
                
                loc_floor, conf_floor = nly.read_loc(Dataframe_floor, scorer_floor, animalID, bodyparts_floor)
                loc_wall,  conf_wall  = nly.read_loc(Dataframe_wall,  scorer_wall,  animalID, bodyparts_wall)
                loc_cross, conf_cross = nly.read_loc(Dataframe_cross, scorer_cross, animalID, bodyparts_cross)
                loc_paint, conf_paint = nly.read_loc_paint(Dataframe_floor,scorer_floor,uniqueID, paint)
                

                thr_cf = 0.95


                fly_tracker_f = nly.tracking_analysis_or_cf(loc_floor,conf_floor,thr_cf)
                fly_tracker_f_n = np.zeros((2,3,2,frame_n))==1
                fly_tracker_f_n[:,:,0,:] = ~fly_tracker_f
                fly_tracker_f_n[:,:,1,:] = ~fly_tracker_f
                loc_floor[fly_tracker_f_n] = np.NaN
                
                fly_tracker_w = nly.tracking_analysis_or_cf(loc_wall, conf_wall,thr_cf)
                fly_tracker_w_n = np.zeros((2,3,2,frame_n))==1
                fly_tracker_w_n[:,:,0,:] = ~fly_tracker_w
                fly_tracker_w_n[:,:,1,:] = ~fly_tracker_w
                loc_wall[fly_tracker_w_n]  = np.NaN

                fly_tracker_c = nly.tracking_analysis_or_cf(loc_cross,conf_cross,thr_cf)
                fly_tracker_c_n = np.zeros((2,4,2,frame_n))==1
                fly_tracker_c_n[:,:,0,:] = ~fly_tracker_c
                fly_tracker_c_n[:,:,1,:] = ~fly_tracker_c
                loc_cross[fly_tracker_c_n]  = np.NaN
                loc_temp_cross          = np.zeros((2,1,2,frame_n))
                loc_temp_cross[:,0,:,:] = np.nanmean(loc_cross, axis=1)
                conf_temp_cross = np.zeros((2,1,frame_n))
                conf_temp_cross[:,0,:] = np.nanmean(conf_cross, axis=1)
                loc_temp          = np.zeros((2,4,2,frame_n))
                loc_temp[:,:,:,:] = np.NaN
                

                    
                    
                    
#                 fly_tracker_f_e = ~nly.tracking_analysis_eql(loc_floor, 1, 2)
#                 idx_frames_e = idx_frames[fly_tracker_f_e]
#                 np.random.shuffle(idx_frames_e)
                
#                 jj = 0
#                 for idx_i in idx_frames_e:
#                     loc_i_f = np.nanmean(loc_floor[:,:,:,idx_i],axis=1)
#                     loc_i_w = np.nanmean(loc_wall[:,:,:,(idx_i)],axis=1)
#                     loc_i_c = np.nanmean(loc_cross[:,:,:,(idx_i)],axis=1)
                    
#                     d_i_fw  = np.sqrt(np.sum(np.square(loc_i_w-loc_i_f),axis=1))
#                     d_i_fc  = np.sqrt(np.sum(np.square(loc_i_c-loc_i_f),axis=1))
                    
#                     d_i_f = np.ones(4)
#                     d_i_f[0:2] = d_i_fw
#                     d_i_f[2:4] = d_i_fc
#                     d_i_f[np.isnan(d_i_f)] = -1
#                     idx_max = np.argmax(d_i_f)
#                     if d_i_f[idx_max] > 0 and d_i_f[idx_max] > 50:
#                         if idx_max==0:
#                             pair = 'f_w1'
#                         elif idx_max==1:
#                             pair = 'f_w2'
#                         elif idx_max==2:
#                             pair = 'f_c1'
#                         elif idx_max==3:
#                             pair = 'f_c2'
                        
#                         loc_f_i = loc_floor[:,:,:,idx_i]
#                         loc_w_i = loc_wall[:,:,:,idx_i]
#                         loc_c_i = loc_temp_cross[:,:,:,idx_i]

#                         cf_f_i = conf_floor[:,:,idx_i]
#                         cf_w_i = conf_wall [:,:,idx_i]
#                         cf_c_i = conf_temp_cross[:,0,idx_i]
#                         image = (output_fileloc + cropped_recording_folder +'/R72/Test/'+well+'_'+format(idx_i,'06d')+'_'+pair)
#                         tkr_dec.save_image_fwc(cap, loc_f_i, loc_w_i, loc_c_i, 
#                                              cf_f_i,  cf_w_i,  cf_c_i, 
#                                        video, idx_i, image, d_xy, bodyparts_floor)
                    
#                     if jj==5:
#                         break
#                     jj += 1
                    
                    
#                 fly_tracker_w_e = ~nly.tracking_analysis_eql(loc_wall, 1, 2)
#                 idx_frames_e = idx_frames[fly_tracker_w_e]
#                 np.random.shuffle(idx_frames_e)
                
#                 jj = 0
#                 for idx_i in idx_frames_e:
                   
#                     loc_i_f = np.nanmean(loc_floor[:,:,:,idx_i],axis=1)
#                     loc_i_w = np.nanmean(loc_wall[:,:,:,(idx_i)],axis=1)
#                     loc_i_c = np.nanmean(loc_cross[:,:,:,(idx_i)],axis=1)
                    
#                     d_i_wf  = np.sqrt(np.sum(np.square(loc_i_w-loc_i_f),axis=1))
#                     d_i_wc  = np.sqrt(np.sum(np.square(loc_i_w-loc_i_c),axis=1))
                    
#                     d_i_w = np.ones(4)
#                     d_i_w[0:2] = d_i_wf
#                     d_i_w[2:4] = d_i_wc
#                     d_i_w[np.isnan(d_i_w)] = -1
#                     idx_max = np.argmax(d_i_w)
#                     if d_i_w[idx_max] >= 0 and d_i_w[idx_max] > 50:
#                         if idx_max==0:
#                             pair = 'w_f1'
#                         elif idx_max==1:
#                             pair = 'w_f2'
#                         elif idx_max==2:
#                             pair = 'w_c1'
#                         elif idx_max==3:
#                             pair = 'w_c2'
                        
#                         loc_f_i = loc_floor[:,:,:,idx_i]
#                         loc_w_i = loc_wall[:,:,:,idx_i]
#                         loc_c_i = loc_temp_cross[:,:,:,idx_i]

#                         cf_f_i = conf_floor[:,:,idx_i]
#                         cf_w_i = conf_wall [:,:,idx_i]
#                         cf_c_i = conf_temp_cross[:,0,idx_i]
#                         image = (output_fileloc + cropped_recording_folder +'/R72/Test/'+well+'_'+format(idx_i,'06d')+'_'+pair)
#                         tkr_dec.save_image_fwc(cap, loc_f_i, loc_w_i, loc_c_i, 
#                                              cf_f_i,  cf_w_i,  cf_c_i, 
#                                        video, idx_i, image, d_xy, bodyparts_floor)
                    
                    if jj == 5:
                        break
                    jj += 1
            else:
                print('no h files')
            ii = ii+1

In [ ]:
filter_status = '_filtered'
# filter_status = ''
ii = 0
for i in range(num_file):
# for i in [1]:
# for i in range(1):
    experiment = experiments_mscore[fileloc_set[i]]
    sexes        = experiment['sex']
    genotypes   = experiment['genotype']
    experiment_type = experiment['events type']
    events = experiment['events']
    arena_circle = experiment['arena circle']
    experiment_index = experiments_index[i]
    well_coordinate = {}
    for j in range(num_well):
#     for j in range(1):
#     for i in [7]:
        sex = sexes[j]
        genotype = genotypes[j]
        
        if sex != -1:
            event_ts   = events[j]
            event_type = experiment_type[j]

            well_arena = arena_circle[j]
            circle = np.mean(well_arena, axis = 0)
            d_xy = [576/2, 576/2]-circle[0:2]-48

            
            parameter_mm2p = (17.4625/circle[2]/2)
            
            n_event = len(event_ts)
        
            well = 'v_'+experiment_index+'_'+videofile_set[j]
            
            hfile_wall = h5_dir+'/'+well+scorer_wall+'_el_filtered.h5'
            my_file_wall = Path(hfile_wall)
            hfile_floor = h5_dir+'/'+well+scorer_floor+'_el_filtered.h5'
            my_file_floor = Path(hfile_floor)
            hfile_cross = h5_dir+'/'+well+scorer_cross+'_el_filtered.h5'
            my_file_cross = Path(hfile_cross)

            hfile_wall = h5_dir+'/'+well+scorer_wall+'_el.h5'
            my_file_wall = Path(hfile_wall)
            hfile_floor = h5_dir+'/'+well+scorer_floor+'_el.h5'
            my_file_floor = Path(hfile_floor)
            hfile_cross = h5_dir+'/'+well+scorer_cross+'_el.h5'
            my_file_cross = Path(hfile_cross)
            
            wall_floor = my_file_wall.is_file() and my_file_floor.is_file()
            wall_floor_cross = wall_floor and my_file_cross.is_file()
            if wall_floor_cross:
                
                video = output_fileloc +converted_folder+fileloc_set[i]+'well_'+videofile_set[j]+'_ts.mp4'
                cap = cv2.VideoCapture(video)
                idx_frames = np.arange(frame_n)
                npz_file_name = h5_dir+'/'+well+'_el'+filter_status+'.npz'
                # Access the loaded arrays
                loaded_data = np.load(npz_file_name)
                loc_temp          = loaded_data['loc_temp']
                fly_tracker_temp  = loaded_data['fly_tracker_temp']
                well_tracker_temp = loaded_data['well_tracker_temp']
                idx_outlier_bp = np.zeros(frame_n)>0
                
                for i_fly in range(n_ID):
                    loc_f_h = loc_temp[i_fly,0,:,:]
                    loc_f_c = loc_temp[i_fly,1,:,:]
                    loc_f_t = loc_temp[i_fly,2,:,:]

                    d_f_hc = np.sqrt(np.sum(np.square(loc_f_h-loc_f_c),axis=0))
                    d_f_ht = np.sqrt(np.sum(np.square(loc_f_h-loc_f_t),axis=0))
                    d_f_ct = np.sqrt(np.sum(np.square(loc_f_c-loc_f_t),axis=0))

                    outlier_bp = np.zeros((3,frame_n))>0

                    outlier_bp[0,:] = d_f_hc > (1.8/parameter_mm2p)
                    outlier_bp[1,:] = d_f_ht > (3.5/parameter_mm2p)
                    outlier_bp[2,:] = d_f_ct > (1.8/parameter_mm2p)
                    
                    idx_outlier_bp = np.logical_or(idx_outlier_bp,np.sum(outlier_bp,axis=0)==2)
                    

                    
                print(well,np.sum(idx_outlier_bp))
                idx_frames_e = idx_frames[idx_outlier_bp]
                np.random.shuffle(idx_frames_e)
                
                jj = 0
                for idx_i in idx_frames_e:

                    image = (output_fileloc + cropped_recording_folder +'/R72/Test/'+well+'_'+format(idx_i,'06d'))
                    loc_temp_i         = loc_temp[:,:,:,idx_i]
                    fly_tracker_temp_i = fly_tracker_temp[:,:,idx_i]
                    tkr_dec.save_image_temp(cap, loc_temp_i, fly_tracker_temp_i, 
                                    video, idx_i, image, d_xy, bodyparts_floor)

                    
                    if jj==5:
                        break
                    jj += 1
            else:
                print('no h files')
            ii = ii+1